<a href="https://colab.research.google.com/github/thazin31086/GitHub_NeturalNetworks/blob/master/IssueReport%20Similarity%20Grouping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import nltk
import re
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation

# tensroflow hub module for Universal sentence Encoder 
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)

def get_features(texts):
    if type(texts) is str:
        texts = [texts]
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return sess.run(embed(texts))

def remove_stopwords(stop_words, tokens):
    res = []
    for token in tokens:
        if not token in stop_words:
            res.append(token)
    return res

def process_text(text):
    text = text.encode('ascii', errors='ignore').decode()
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'#+', ' ', text )
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
    #text = re.sub(r"\'s", " ", text)     text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"isn't", "is not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text


def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_list = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, 'v')
        if lemma == token:
            lemma = lemmatizer.lemmatize(token)
        lemma_list.append(lemma)
    # return [ lemmatizer.lemmatize(token, 'v') for token in tokens ]     return lemma_list


def process_all(text):
    text = process_text(text)
    return ' '.join(remove_stopwords(stop_words, text.split()))

def cosine_similarity(v1, v2):
    mag1 = np.linalg.norm(v1)
    mag2 = np.linalg.norm(v2)
    if (not mag1) or (not mag2):
        return 0
    return np.dot(v1, v2) / (mag1 * mag2)

def test_similarity(text1, text2):
    vec1 = get_features(text1)[0]
    vec2 = get_features(text2)[0]
    print(vec1.shape)
    return cosine_similarity(vec1, vec2)

def semantic_search(query, data, vectors):
    query = process_text(query)
    print("Extracting features...")
    query_vec = get_features(query)[0].ravel()
    res = []
    for i, d in enumerate(data):
        qvec = vectors[i].ravel()
        sim = cosine_similarity(query_vec, qvec)
        res.append((sim, d[:100], i))
    return sorted(res, key=lambda x : x[0], reverse=True)

def unique_words(sentence):
    return set(sentence.lower().split())
 

In [0]:
data = [
    "Avoid crash on concat on structs with ToString member", #https://github.com/dotnet/roslyn/pull/38860/commits
    "Enum implicit cast to string fails when element is named ToString", #https://github.com/dotnet/roslyn/issues/40256
    "Enum with ToString member crashes in string concatenation", #https://github.com/dotnet/roslyn/issues/38858
    "Visual Studio 2019 crashing when click RMB on rule in Analyzers' dependencies",#https://github.com/dotnet/roslyn/issues/40720
    "Crash on right-click a analyse rule in Solution-Explorer", #https://github.com/dotnet/roslyn/issues/36304
    "Handle lazy loading of analyzer command handlers", #https://github.com/dotnet/roslyn/pull/36740
]

In [5]:
data_processed = list(map(process_text, data))
BASE_VECTORS = get_features(data)
#semantic_search("Avoid crash on concat on structs with ToString member", data_processed, BASE_VECTORS)
semantic_search("Visual Studio 2019 crashing when click RMB on rule in Analyzers' dependencies", data_processed, BASE_VECTORS)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Extracting features...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[(0.98528624,
  'visual studio crashing when click rmb on rule in analyzers dependencies',
  3),
 (0.699809, 'avoid crash on concat on structs with tostring member', 0),
 (0.67198277, 'enum with tostring member crashes in string concatenation', 2),
 (0.64385056, 'crash on right click a analyse rule in solution explorer', 4),
 (0.63877857,
  'enum implicit cast to string fails when element is named tostring',
  1),
 (0.6170363, 'handle lazy loading of analyzer command handlers', 5)]

In [7]:
#Compare with Code
issuereport_uniquewordlist = unique_words(' '.join(data[3:6]))
issuereport_cluser1 = ' '.join(issuereport_uniquewordlist)
codedata = ['project should other severity link get changed add item name items with hierarchy create checked show help initialize rule active context handler new folder enabled any controller menu diagnostics copy file update diagnostic analyzer open analyzers selected command remove set', 
            'Analyzer Command Handler Tests Traits Features Diagnostics Diagnostic Context Menu Controller Solution Explorer']
codedata_processed = list(map(process_text, codedata))          
code_BASE_VECTORS = get_features(codedata)

print(issuereport_cluser1)
semantic_search(issuereport_cluser1, codedata_processed, code_BASE_VECTORS)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


right-click crashing when crash in 2019 of loading click studio a on command analyzer dependencies visual solution-explorer rmb analyzers' handle lazy handlers rule analyse
Extracting features...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[(0.7856611,
  'analyzer command handler tests traits features diagnostics diagnostic context menu controller soluti',
  1),
 (0.64409107,
  'project should other severity link get changed add item name items with hierarchy create checked sho',
  0)]